In [2]:
import pybullet as p
import time
import pybullet_data
import os, inspect
import numpy as np
import copy
import math
import random

In [3]:
class braccio_arm:

  def __init__(self, urdfRootPath=pybullet_data.getDataPath(), timeStep=0.01):
    self.urdfRootPath = urdfRootPath
    self.timeStep = timeStep
    self.maxVelocity = .35
    self.maxForce = 200.
    self.fingerAForce = 2
    self.fingerBForce = 2.5
    self.fingerTipForce = 2
    self.useInverseKinematics = 1
    self.useSimulation = 1
    self.useNullSpace = 1
    self.useOrientation = 0
    self.baEndEffectorIndex = 5
    self.baGripperIndex = 5
    self.baFingerIndexL = 6
    self.baFingerIndexR = 7 #check urdf file with the index number
    # lower limits for null space
    self.ll = [-.967, -2, -2.96, 0.19, -2.96, -2.09, -3.05]
    #self.ll = [-10, -10, -10, -10, -10, -10, -10]

    # upper limits for null space
    self.ul = [.967, 2, 2.96, 2.29, 2.96, 2.09, 3.05]
    #self.ul = [10, 10, 10, 10, 10, 10, 10]
    # joint ranges for null space
    self.jr = [5.8, 4, 5.8, 4, 5.8, 4, 6]
    #self.jr = [10, 10, 10, 10, 10, 10, 10]
    # restposes for null space
    self.rp = [0, 0, 0, 0.5 * math.pi, 0, -math.pi * 0.5 * 0.66, 0]
    # joint damping coefficents
    self.jd = None
    self.reset()

  def reset(self):

    robot = p.loadURDF("/home/jessie/internship/model/braccio_arm_clean.urdf") 
    self.baUid = robot
    p.resetBasePositionAndOrientation(self.baUid, [0.0, 0.0, 0.0], # position of robot, GREEN IS Y AXIS
                                      [0.000000, 0.000000, 0.000000, 1.000000]) # direction of robot
    self.jointPositions = [
        0.0, 1.4412599, 0.39843294, 2.1354597, 
        0.52981985, 0.30501562, 0.89014286
        ] # 7 joints random reset joint angles



    self.numJoints = p.getNumJoints(self.baUid)
    for jointIndex in range(self.numJoints):
        p.resetJointState(self.baUid, jointIndex, self.jointPositions[jointIndex])
        p.setJointMotorControl2(self.baUid,
                              jointIndex,
                              p.POSITION_CONTROL,
                              targetPosition=self.jointPositions[jointIndex],
                              force=self.maxForce)

    self.endEffectorPos = [0.52981985, 0.30501562, 0.89014286]# [0.4317596244807792, 0.1470447615125933, 0.2876258566462587] #TODO
    
    self.endEffectorAngle = 0 #0.02 #TODO

    self.motorNames = []
    self.motorIndices = []

    for i in range(self.numJoints):
      jointInfo = p.getJointInfo(self.baUid, i)
      qIndex = jointInfo[3]
      if qIndex > -1:
        self.motorNames.append(str(jointInfo[1]))
        self.motorIndices.append(i)

  def getActionDimension(self):
    if (self.useInverseKinematics):
      return len(self.motorIndices)
    return 6  #position x,y,z and roll/pitch/yaw euler angles of end effector

  def getObservationDimension(self):
    return len(self.getObservation())

  def getObservation(self):
    observation = []
    # state for gripper(end effector)
    state = p.getLinkState(self.baUid, self.baGripperIndex)
    pos = state[0]
    orn = state[1] #Cartesian orientation of center of mass, in quaternion [x,y,z,w]

    euler = p.getEulerFromQuaternion(orn)

    observation.extend(list(pos))
    observation.extend(list(euler))

    return observation

  def applyAction(self, motorCommands):


    #print ("self.numJoints")
    #print (self.numJoints)
    if (self.useInverseKinematics):

      dx = motorCommands[0]
      dy = motorCommands[1]
      dz = motorCommands[2]
      da = motorCommands[3]
      fingerAngle = motorCommands[4]
      state = p.getLinkState(self.baUid, self.baEndEffectorIndex) # returns 1. center of mass cartesian coordinates, 2. rotation around center of mass in quaternion
      actualEndEffectorPos = state[0]


      self.endEffectorPos[0] = self.endEffectorPos[0] + dx

      self.endEffectorPos[1] =  self.endEffectorPos[1] +  dy

      self.endEffectorPos[2] = self.endEffectorPos[2] +  dz
  #
      self.endEffectorAngle = self.endEffectorAngle + da
      pos = actualEndEffectorPos
      orn = p.getQuaternionFromEuler([0, -math.pi, 0])  # -math.pi,yaw]) 
      if (self.useNullSpace == 1): 
        if (self.useOrientation == 1):
          jointPoses = p.calculateInverseKinematics(self.baUid, self.baEndEffectorIndex, pos,
                                                    orn, self.ll, self.ul, self.jr, self.rp)
        else:
          jointPoses = p.calculateInverseKinematics(self.baUid,
                                                    self.baEndEffectorIndex,
                                                    pos)
                                                    # lowerLimits=self.ll,
                                                    # upperLimits=self.ul,
                                                    # jointRanges=self.jr,
                                                    # restPoses=self.rp)
      else:
        if (self.useOrientation == 1):
          jointPoses = p.calculateInverseKinematics(self.baUid,
                                                    self.baEndEffectorIndex,
                                                    pos,
                                                    orn,
                                                    jointDamping=self.jd)
        else:
          jointPoses = p.calculateInverseKinematics(self.baUid, self.baEndEffectorIndex, pos)

      if (self.useSimulation):
        for i in range(self.baEndEffectorIndex):

          p.setJointMotorControl2(bodyUniqueId=self.baUid,
                                  jointIndex=i,
                                  controlMode=p.POSITION_CONTROL,
                                  targetPosition=jointPoses[i],
                                  targetVelocity=0,
                                  force=self.maxForce,
                                  maxVelocity=self.maxVelocity,
                                  positionGain=0.3,
                                  velocityGain=1)
      else:
        #reset the joint state (ignoring all dynamics, not recommended to use during simulation)
        for i in range(self.numJoints):
          p.resetJointState(self.baUid, i, jointPoses[i])

      #fingers
      p.setJointMotorControl2(self.baUid,  #control of fingers
                          5,
                          p.POSITION_CONTROL,
                          targetPosition=-fingerAngle/4.,
                          force=self.fingerTipForce)

      p.setJointMotorControl2(self.baUid,
                          6,
                          p.POSITION_CONTROL,
                          targetPosition=fingerAngle/4.,
                          force=self.fingerTipForce)


    else:
      for action in range(len(motorCommands)):
        motor = self.motorIndices[action]
        p.setJointMotorControl2(self.baUid,
                                motor,
                                p.POSITION_CONTROL,
                                targetPosition=motorCommands[action],
                                force=self.maxForce)

  def grasping(self):

    p.setJointMotorControl2(self.baUid,
                          5,
                          p.POSITION_CONTROL,
                          targetPosition=0,
                          force=self.fingerTipForce)
    p.setJointMotorControl2(self.baUid,
                          6,
                          p.POSITION_CONTROL,
                          targetPosition=0,
                          force=self.fingerTipForce)



if __name__ == '__main__':


    physicsClient = p.connect(p.GUI)#or p.DIRECT for non-graphical version
    braccio_arm_test = braccio_arm()
    #p.setGravity(0,0,-9.81)
    braccio_arm_test.applyAction([0.67, 0.2, 0.05,1,0.1])
    for i in range (10000):
        p.stepSimulation()
        time.sleep(1./240.0)
    #braccio_arm_test.grasping()
    p.disconnect()



startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=VMware, Inc.
GL_RENDERER=SVGA3D; build: RELEASE;  LLVM;
GL_VERSION=4.1 (Core Profile) Mesa 22.0.1
GL_SHADING_LANGUAGE_VERSION=4.10
pthread_getconcurrency()=0
Version = 4.1 (Core Profile) Mesa 22.0.1
Vendor = VMware, Inc.
Renderer = SVGA3D; build: RELEASE;  LLVM;
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = VMware, Inc.
ven = VMware, Inc.
numActiveThreads = 0
stopping threads
destroy semaphore
semaphore destroyed
Thread with taskId 0 exiting
Thread TERMINATED
destroy main semaphore
main semaphore des

In [ ]:
# Code base from pybullet examples https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_envs/bullet/ kuka_diverse_object_gym_env.py

import os, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
print("current_dir=" + currentdir)
os.sys.path.insert(0, currentdir)

import gym
from gym import spaces
from gym.utils import seeding
import numpy as np
import time
import random
from pkg_resources import parse_version
largeValObservation = 100

RENDER_HEIGHT = 720
RENDER_WIDTH = 960
maxSteps = 700
Dv = 0.004


class braccio_arm_gym(gym.Env):
  #metadata = {'render.modes': ['human', 'rgb_array'], 'video.frames_per_second': 50}

  def __init__(self,
               urdfRoot=pybullet_data.getDataPath(),
               actionRepeat=1,
               isEnableSelfCollision=True,
               renders=False,
               isDiscrete=False,
               maxSteps=maxSteps):
    self._isDiscrete = isDiscrete
    self._timeStep = 1. / 240.
    self._urdfRoot = urdfRoot
    self._actionRepeat = actionRepeat
    self._isEnableSelfCollision = isEnableSelfCollision
    self._observation = []
    self._envStepCounter = 0
    self._renders = renders
    self._maxSteps = maxSteps
    self.terminated = 0
    self._cam_dist = 1.3
    self._cam_yaw = 180
    self._cam_pitch = -40

    self._p = p
    if self._renders:
      cid = p.connect(p.SHARED_MEMORY)
      if (cid < 0):
        cid = p.connect(p.GUI)
      p.resetDebugVisualizerCamera(1.3, 180, -41, [0.52, -0.2, -0.33])
    else:
      p.connect(p.DIRECT)
    self.seed()
    self.reset()
    observationDim = len(self.getExtendedObservation())
    #print(observationDim)

    observation_high = np.array([largeValObservation] * observationDim)
    if (self._isDiscrete):
      self.action_space = spaces.Discrete(7) #?7
    else:
      action_dim = 3
      self._action_bound = 1
      action_high = np.array([self._action_bound] * action_dim)
      # print(action_high)
      self.action_space = spaces.Box(-action_high, action_high)
    self.observation_space = spaces.Box(-observation_high, observation_high)
    self.viewer = None

  def reset(self):
    self.terminated = 0
    p.resetSimulation()
    p.setPhysicsEngineParameter(numSolverIterations=150)
    p.setTimeStep(self._timeStep)
    p.loadURDF(os.path.join(self._urdfRoot, "plane.urdf"), [0, 0, -1])

    self.tableUid = p.loadURDF(os.path.join(self._urdfRoot, "table/table.urdf"), 0.5000000, 0.00000, -.640000,
               0.000000, 0.000000, 0.0, 1.0)

    xpos = 0.55 + 0.12 * random.random()
    ypos = 0 + 0.2 * random.random()
    ang = 3.14 * 0.5 +1.5 #* random.random()
    orn = p.getQuaternionFromEuler([0, 0, ang])
    self.blockUid = p.loadURDF(os.path.join(self._urdfRoot, "jenga/jenga.urdf"), xpos, ypos, 0.1,
                               orn[0], orn[1], orn[2], orn[3])

    p.setGravity(0, 0, -10)
    self._ba = braccio_arm(urdfRootPath=self._urdfRoot, timeStep=self._timeStep)
    self._envStepCounter = 0
    p.stepSimulation()
    self._observation = self.getExtendedObservation()
    return np.array(self._observation)

  def __del__(self):
    p.disconnect()

  def seed(self, seed=None):
    self.np_random, seed = seeding.np_random(seed)
    return [seed]

  def getExtendedObservation(self):
    self._observation = self._ba.getObservation()
    gripperState = p.getLinkState(self._ba.baUid, self._ba.baFingerIndexL)
    gripperStateR = p.getLinkState(self._ba.baUid, self._ba.baFingerIndexR)

    gripperPos = gripperState[0]
    gripperOrn = gripperState[1]
    gripperPosR = gripperStateR[0]
    gripperOrnR = gripperStateR[1]
    blockPos, blockOrn = p.getBasePositionAndOrientation(self.blockUid)

    invGripperPos, invGripperOrn = p.invertTransform(gripperPos, gripperOrn)
    invGripperPosR, invGripperOrnR = p.invertTransform(gripperPosR, gripperOrnR)

    gripperMat = p.getMatrixFromQuaternion(gripperOrn)
    gripperMatR = p.getMatrixFromQuaternion(gripperOrnR)

    blockPosInGripper, blockOrnInGripper = p.multiplyTransforms(invGripperPos, invGripperOrn,
                                                                blockPos, blockOrn)
    blockPosInGripperR, blockOrnInGripperR = p.multiplyTransforms(invGripperPosR, invGripperOrnR,
                                                                blockPos, blockOrn)
    blockEulerInGripper = p.getEulerFromQuaternion(blockOrnInGripper)
    blockEulerInGripperR = p.getEulerFromQuaternion(blockOrnInGripperR)

    #we return the relative x,y position and euler angle of block in gripper space
    blockInGripperPosXYEulZ = [blockPosInGripper[0], blockPosInGripper[1], blockEulerInGripper[2]]
    blockInGripperPosXYEulZR = [blockPosInGripperR[0], blockPosInGripperR[1], blockEulerInGripper[2]]

    self._observation.extend(list(blockInGripperPosXYEulZ))
    self._observation.extend(list(blockInGripperPosXYEulZR))

    return self._observation


  def step(self, action):
    if (self._isDiscrete):
      dv = Dv
      dx = [0, -dv, dv, 0, 0, 0, 0][action]
      dy = [0, 0, 0, -dv, dv, 0, 0][action]
      da = [0, 0, 0, 0, 0, -0.05, 0.05][action]
      f = 0.15
      realAction = [dx, dy, -0.0005, da, f]
    else:
      # print("action[0]=", str(action[0]))
      dv = Dv
      dx = action[0] * dv
      dy = action[1] * dv
      da = action[2] * 0.05
      f = 0.15
      realAction = [dx, dy, -0.0005, da, f]
    return self.step2(realAction)

  def step2(self, action):
    for i in range(self._actionRepeat):
      self._ba.applyAction(action)
      p.stepSimulation()
      if self._termination():
        break
      self._envStepCounter += 1
    if self._renders:
      time.sleep(self._timeStep)
    self._observation = self.getExtendedObservation()

    done = self._termination()
    npaction = np.array([
        action[3]
    ])  #only penalize rotation until learning works well [action[0],action[1],action[3]])
    actionCost = np.linalg.norm(npaction) * 10.
    #print("actionCost")
    #print(actionCost)
    reward = self._reward() - actionCost
    #print("reward")
    #print(reward)

    #print("len=%r" % len(self._observation))

    return np.array(self._observation), reward, done, {}

  def render(self, mode="rgb_array", close=False):
    if mode != "rgb_array":
      return np.array([])

    base_pos, orn = self._p.getBasePositionAndOrientation(self._ba.baUid)
    view_matrix = self._p.computeViewMatrixFromYawPitchRoll(cameraTargetPosition=base_pos,
                                                            distance=self._cam_dist,
                                                            yaw=self._cam_yaw,
                                                            pitch=self._cam_pitch,
                                                            roll=0,
                                                            upAxisIndex=2)
    proj_matrix = self._p.computeProjectionMatrixFOV(fov=60,
                                                     aspect=float(RENDER_WIDTH) / RENDER_HEIGHT,
                                                     nearVal=0.1,
                                                     farVal=100.0)
    (_, _, px, _, _) = self._p.getCameraImage(width=RENDER_WIDTH,
                                              height=RENDER_HEIGHT,
                                              viewMatrix=view_matrix,
                                              projectionMatrix=proj_matrix,
                                              renderer=self._p.ER_BULLET_HARDWARE_OPENGL)
    #renderer=self._p.ER_TINY_RENDERER)

    rgb_array = np.array(px, dtype=np.uint8)
    rgb_array = np.reshape(rgb_array, (RENDER_HEIGHT, RENDER_WIDTH, 4))

    rgb_array = rgb_array[:, :, :3]
    return rgb_array

  def _termination(self):
    #print (self._tm700.endEffectorPos[2])
    state = p.getLinkState(self._ba.baUid, self._ba.baEndEffectorIndex)
    actualEndEffectorPos = state[0]

    if (self.terminated or self._envStepCounter > self._maxSteps):
      self._observation = self.getExtendedObservation()
      return True
    maxDist = 0.006
    closestPoints = p.getClosestPoints(self.tableUid, self._ba.baUid, maxDist, -1, self._ba.baFingerIndexL)

    if (len(closestPoints)):  #(actualEndEffectorPos[2] <= -0.43):
      self.terminated = 1

      #print("terminating, closing gripper, attempting grasp")
      #start grasp and terminate
      fingerAngle = 0.15
      for i in range(1000):
        graspAction = [0, 0, 0.0005, 0, fingerAngle]
        self._ba.applyAction(graspAction)
        p.stepSimulation()
        fingerAngle = fingerAngle - (0.3 / 100.)
        if (fingerAngle < 0):
          fingerAngle = 0

      for i in range(10000):
        graspAction = [0, 0, 0.001, 0, fingerAngle]
        self._ba.applyAction(graspAction)
        p.stepSimulation()
        blockPos, blockOrn = p.getBasePositionAndOrientation(self.blockUid)
        if (blockPos[2] > 0.23):
          #print("BLOCKPOS!")
          #print(blockPos[2])
          break
        state = p.getLinkState(self._ba.baUid, self._ba.baEndEffectorIndex)
        actualEndEffectorPos = state[0]
        if (actualEndEffectorPos[2] > 0.5):
          break

      self._observation = self.getExtendedObservation()
      return True
    return False

  def _reward(self):

    #rewards is height of target object
    blockPos, blockOrn = p.getBasePositionAndOrientation(self.blockUid)
    closestPoints1 = p.getClosestPoints(self.blockUid, self._ba.baUid, 10, -1,
                                       self._ba.baFingerIndexL)
    closestPoints2 = p.getClosestPoints(self.blockUid, self._ba.baUid, 10, -1,
                                       self._ba.baFingerIndexR) # id of object a, id of object b, max. separation, link index of object a (base is -1), linkindex of object b

    # fingerL = p.getLinkState(self._tm700.tm700Uid, self._tm700.tmFingerIndexL)
    # fingerR = p.getLinkState(self._tm700.tm700Uid, self._tm700.tmFingerIndexR)
    # print('infi', np.mean(list(fingerL[0])))


    reward = -1000

    # print(closestPoints1[0][8])
    closestPoints = closestPoints1[0][8]
    numPt = len(closestPoints1)
    #print(numPt)
    if (numPt > 0):
      #print("reward:")
      # reward = -1./((1.-closestPoints1[0][8] * 100 + 1. -closestPoints2[0][8] * 100 )/2)
      reward = -((closestPoints1[0][8])**2 + (closestPoints2[0][8])**2 )*(1/0.17849278457978357)
      # reward = 1/((abs(closestPoints1[0][8])   + abs(closestPoints2[0][8])*10 )**2 / 2)
      # reward = 1/closestPoints1[0][8]+1/closestPoints2[0][8]
    if (blockPos[2] > 0.2):
      reward = reward + 1000
      print("successfully grasped a block!!!")
      #print("self._envStepCounter")
      #print(self._envStepCounter)
      #print("self._envStepCounter")
      #print(self._envStepCounter)
      #print("reward")
      #print(reward)
    # print("reward")
    # print(reward)
    return reward

  if parse_version(gym.__version__) < parse_version('0.9.6'):
    _render = render
    _reset = reset
    _seed = seed
    _step = step


if __name__ == '__main__':

# datapath = pybullet_data.getDataPath()
  p.connect(p.GUI, options="--opencl2")
  #p.setAdditionalSearchPath(datapath)
  test =braccio_arm_gym()
  for i in range(10000):
    # test.step2([0.55, 0.2, 0.05,0,0])
    p.stepSimulation()
    time.sleep(1. / 240.0)

  time.sleep(50)